In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append("/workspaces/MambaLinearCode")
os.chdir("/workspaces/MambaLinearCode")


In [ ]:
from configuration import Code, Config
from dataset import get_generator_and_parity
import torch
import os
import logging

def code_from_hint(hint,):
    code_files = os.listdir(CODES_PATH)
    code_files = [f for f in code_files if hint in f][0]
    print(code_files)
    code_n = int(code_files.split('_')[1][1:])
    code_k = int(code_files.split('_')[-1][1:].split('.')[0])
    code_type = code_files.split('_')[0]
    code = Code(code_n, code_k, code_type)
    return code

OUTPUT_PATH = ".output/"
CODES_PATH = "codes/"
example_code = code_from_hint("LDPC_N49_K24")
G,H = get_generator_and_parity(example_code, standard_form=True)
example_code.generator_matrix = torch.from_numpy(G).transpose(0,1).long()
example_code.pc_matrix = torch.from_numpy(H).long()

os.makedirs(OUTPUT_PATH, exist_ok=True)
config = Config(
    code=example_code,
    d_model=64,
    d_state=64,
    path=OUTPUT_PATH,
    N_dec=2,
    warmup_lr=1.0e-4,
    lr=1.0e-4
)

handlers = [
        logging.FileHandler(os.path.join(OUTPUT_PATH, 'logging.txt')),
        logging.StreamHandler()
    ]
logging.basicConfig(level=logging.INFO, format='%(message)s',
                    handlers=handlers)

In [ ]:
from mamba_ssm import Mamba
from dataset import EbN0_to_std, ECC_Dataset, train, test, sign_to_bin
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn.functional as F
from torch.nn import ModuleList, LayerNorm
import copy

device = "cuda"

def clones(module, N):
    return ModuleList([copy.deepcopy(module) for _ in range(N)])

class EncoderLayer(torch.nn.Module):
    def __init__(self, config: Config, length) -> None:
        super().__init__()
        self.mamba = Mamba(
            d_model=config.d_model,
            d_state=config.d_state
        )
        self.norm = LayerNorm((length, config.d_model))
    
    def forward(self, x):
        o = self.mamba.forward(x)
        return self.norm(o)

class ECCM(torch.nn.Module):
    def __init__(self, config: Config) -> None:
        super().__init__()
        self.n = config.code.n
        self.syndrom_length = config.code.pc_matrix.size(0)
        self.src_embed = torch.nn.Parameter(torch.ones(
            (self.n + self.syndrom_length, config.d_model)))
        self.mamba: ModuleList = clones(EncoderLayer(config, (self.n + self.syndrom_length)), config.N_dec)
        self.resize_output_dim = torch.nn.Linear(config.d_model, 1)
        self.resize_output_length = torch.nn.Linear(self.n + self.syndrom_length, self.n)
    
    def forward(self, magnitude, syndrome):
        emb = torch.cat([magnitude, syndrome], -1).unsqueeze(-1)
        out: torch.Tensor = self.src_embed.unsqueeze(0) * emb
        for sublayer in self.mamba:
            out: torch.Tensor = sublayer.forward(out)
        
        out: torch.Tensor = self.resize_output_length(out.swapaxes(-2,-1))
        out: torch.Tensor = self.resize_output_dim(out.swapaxes(-2,-1))
        out: torch.Tensor = out.squeeze(-1)
        return out

    def loss(self, z_pred, z2, y):
        loss = F.binary_cross_entropy_with_logits(
            z_pred, sign_to_bin(torch.sign(z2)))
        x_pred = sign_to_bin(torch.sign(-z_pred * torch.sign(y)))
        return loss, x_pred


model = ECCM(config=config).to("cuda")

def train_model(args: Config, model):
    code = args.code
    initial_lr = args.warmup_lr
    device = "cuda" if torch.cuda.is_available() else "cpu"
    optimizer = Adam(model.parameters(), lr=args.warmup_lr)
    

    #################################
    EbNo_range_test = range(4, 7)
    EbNo_range_train = range(2, 8)
    std_train = [EbN0_to_std(ii, code.k / code.n) for ii in EbNo_range_train]
    std_test = [EbN0_to_std(ii, code.k / code.n) for ii in EbNo_range_test]
    train_dataloader = DataLoader(ECC_Dataset(code, std_train, len=args.batch_size * 1000, zero_cw=True), batch_size=int(args.batch_size),
                                  shuffle=True, num_workers=args.workers)
    test_dataloader_list = [DataLoader(ECC_Dataset(code, [std_test[ii]], len=int(args.test_batch_size), zero_cw=False),
                                       batch_size=int(args.test_batch_size), shuffle=False, num_workers=args.workers) for ii in range(len(std_test))]
    #################################

    best_loss = float('inf')
    # for epoch in range(1,3):
    #     loss, ber, fer = train(model, device, train_dataloader, optimizer,
    #                            epoch, LR=initial_lr, config=args)
    #     if loss < best_loss:
    #         best_loss = loss
    #         torch.save(model.state_dict(), os.path.join(args.path, 'best_model'))
    
    for param_group in optimizer.param_groups:
        param_group['lr'] = args.lr
    
    scheduler = CosineAnnealingLR(optimizer, T_max=1000, eta_min=args.eta_min)
    for epoch in range(1, args.epochs + 1):
        loss, ber, fer = train(model, device, train_dataloader, optimizer,
                               epoch, LR=scheduler.get_last_lr()[0], config=args)
        scheduler.step()
        if loss < best_loss:
            best_loss = loss
            torch.save(model.state_dict(), os.path.join(args.path, 'best_model'))

        if epoch % 200 == 0:
            test(model, device, test_dataloader_list, EbNo_range_test)
    return model

train_model(config, model)

In [ ]:
config.d_model